<a href="https://colab.research.google.com/github/DmitriyValetov/nlp_course_project/blob/master/rs_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RossiyaSegodnya Dataset for PyTorch
[github repository](https://github.com/RossiyaSegodnya/ria_news_dataset)

Full dataset 1003869 news 

https://github.com/RossiyaSegodnya/ria_news_dataset/raw/master/ria.json.gz

20 news 

https://raw.githubusercontent.com/RossiyaSegodnya/ria_news_dataset/master/ria_20.json

1000 news

https://raw.githubusercontent.com/RossiyaSegodnya/ria_news_dataset/master/ria_1k.json

normalized divided into sentences dataset (use second block)

https://drive.google.com/uc?id=1-UtATnzLE809Vi6RLgy3GRHX2TXRzhd6&export=download

# Download

## Raw

In [19]:
import requests
import os

url = "https://raw.githubusercontent.com/RossiyaSegodnya/ria_news_dataset/master/ria_20.json"
# url = "https://raw.githubusercontent.com/RossiyaSegodnya/ria_news_dataset/master/ria_1k.json"
# url = "https://github.com/RossiyaSegodnya/ria_news_dataset/raw/master/ria.json.gz"
fn, ext = os.path.splitext(os.path.basename(url))
print(f'downloading {fn + ext}')
r = requests.get(url) 
with open(fn + ext, 'wb') as f:
  f.write(r.content)
if ext == '.gz':  # requests should decompress .gz by default but don't...
  print(f'decompressing from {fn + ext} to {fn}')
  import gzip
  import shutil
  with gzip.open(fn + ext, 'rb') as gz_file:
    with open(fn, 'wb') as json_file:
      shutil.copyfileobj(gz_file, json_file)

downloading ria_20.json


## Normalized and Tokenized

In [9]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
file_id = '1-UtATnzLE809Vi6RLgy3GRHX2TXRzhd6'
downloaded = drive.CreateFile({'id': file_id})
downloaded.FetchMetadata()
fn = downloaded.metadata['title']
print(f'downloading: {fn}')
downloaded.GetContentFile(fn)

downloading: norm_sents_ria.json.gz


# Normalize and Tokenize
Skip this if you loaded a normalized dataset

In [0]:
import nltk
nltk.download('punkt')  # for sentence tokenization

In [0]:
import json
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

# Files are not clear json files. They contain json strings line by line.
fn = 'ria.json'
norm_fn = f'norm_sents_{fn}'  # news divided into normalized sentences
with open(fn) as f:
  with open(norm_fn, 'a+') as nf:
    for line in tqdm(f, total=1003869):  # 1003869 - news at full dataset
      n = json.loads(line)
      text = BeautifulSoup(n['text']).get_text()
      title = BeautifulSoup(n['title']).get_text()
      text_ss = sent_tokenize(text)  # text sentences
      title_ss = sent_tokenize(title)  # title sentences
      norm_text_ss = [' '.join(w for w in word_tokenize(s) if w.isalnum()) for s in text_ss]
      norm_title_ss = [' '.join(w for w in word_tokenize(s) if w.isalnum()) for s in title_ss]
      json_str = json.dumps({'text': norm_text_ss, 'title': norm_title_ss}, ensure_ascii=False)
      nf.write(json_str + '\n')
with open(norm_fn) as nf: 
  print(json.loads(next(nf)))  # print first news

In [0]:
import gzip
import shutil

comp_fn = norm_fn + '.gz'
# compress normalized dataset
print(f'compressing {norm_fn} to {comp_fn}')
with gzip.open(comp_fn + '.gz', 'wb') as gz_file:
  with open(norm_fn, 'rb') as json_file:
    shutil.copyfileobj(json_file, gz_file)

# check
with gzip.open(comp_fn, 'rb') as gz_file:
  print(json.loads(next(gz_file)))

# Process

In [17]:
import gzip
import json

comp_fn = 'norm_sents_ria.json.gz'

# check
with gzip.open(comp_fn, 'rb') as gz_file:
  print(json.loads(next(gz_file)))

{'text': ['москва 31 янв риа новости', 'большая часть из 33 детей которых граждане сша пытались вывезти из гаити в организованный в доминиканской республике приют не являются сиротами сообщает в воскресенье агентство франс пресс со ссылкой на заявление представителя международной организации детские деревни sos sos children village оказывающей помощь детям оставшимся без родителей как заявила агентству патрисия варгас patricia vargas курирующая программы детских деревень sos в центральной америке мексике и на карибах поговорив с детьми она выяснила что родители многих из них живы', 'некоторые дети смогли назвать свои домашние адреса и номера телефонов что дает возможность связаться с их родителями', 'в это воскресенье гаитянская полиция задержала десятерых граждан сша подозреваемых в попытке без разрешения вывезти более 30 детей в доминиканскую республику', 'представитель баптистской церкви в городе меридиан американского штата айдахо шон лэнкфорд sean lankford заявил что задержанные п